In [1]:
import signac
import numpy as np
import matplotlib as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd
import scipy
from scipy import signal
import time
import os

In [7]:
ptb7_proj = signac.get_project("ptb7-project")
itic_proj = signac.get_project("ptb7-project")
ptb7_proj

Project.get_project('/home/emilyepstein/projects/ptb7-itic/ptb7-project')

In [12]:
sweep_jobs

JobsCursor(project=Project.get_project('/home/emilyepstein/projects/ptb7-itic/ptb7-project'), filter={'size': 'small', 'process': 'quench'}, doc_filter=None)

In [23]:
def data_analysis(jobs, function = 'get_rdf_data'()):
    for job in jobs:
        job_path = job.workspace()
        if not job.isfile('rdf-data.txt'):
            rdf_results = rdf_from_gsd(job) 
            txt_file = "{}/rdf-data.txt".format(job.workspace(), atom_types)
            x = rdf.bin_centers
            y = rdf.rdf
            np.savetxt(gsd_file, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    

In [ ]:
state_dict = {"size":"small", "process":"quench"}

job_list = ptb7_proj.find_jobs(state_dict)

In [27]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions


def rdf_from_gsd(job, atom_types='all', r_max=5, r_min=0.5, nbins=50, start=95):
    gsd_file = "{}/trajectory.gsd".format(job.workspace())
    f = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset=False)
 
    f.close()
    return freud_rdf

#def rdf(job):
#    calc rdf
#    save to a text file
#    save file in jobs workspace
    
    
#def msd(job):
#    slope using last x amount of steps
#    save vale of slope to job state_point_dict


In [26]:
def get_rdf_data(gsd_file, rdf_type):
    data = np.genfromtxt(file, names=True, delimiter=',')
    x = data['{}_x'.format(rdf_type)]
    y = data['{}_y'.format(rdf_type)]
    return x,y